Given a biobrick, return:
1. 1st degree parts network
2. add network to author --> expand all parts created by the author
3. add igem team --> expand all parts created by the igem team

In [ ]:
import duckdb
import networkx as nx
from pyvis.network import Network
from pathlib import Path

In [ ]:
# convert sqlite to duckdb
sqlite_path = "../data/raw/igem_parts_registry.sqlite"
outfile = Path("../data/processed/duck.db")
outfile.parent.mkdir(parents=True, exist_ok=True)
with duckdb.connect(str(outfile)) as conn:        
    try:
        conn.execute(f"ATTACH '{sqlite_path}' (TYPE sqlite); CREATE OR REPLACE TABLE parts as SELECT * FROM igem_parts_registry.parts")
    except duckdb.duckdb.BinderException:
        conn.execute(f"CREATE OR REPLACE TABLE parts as SELECT * FROM igem_parts_registry.parts")
    conn.commit()
    conn.close()

In [ ]:
def load_file(db: str = "duck.db", sqlite_path: str = "data/raw/igem_parts_registry.sqlite", table_name: str = "igem_parts_registry.parts"):
    with duckdb.connect(db) as conn:
        conn.execute(f"""ATTACH '{sqlite_path}' (TYPE sqlite); SELECT * FROM igem_parts_registry.parts;""")
    return True

In [ ]:
def load_parts_table(db: str = "duck.db", sqlite_path: str = "data/raw/igem_parts_registry.sqlite"):
    with duckdb.connect(db) as conn:        
        try:
            df_parts = duckdb.sql(f"ATTACH '{sqlite_path}' (TYPE sqlite); SELECT * FROM igem_parts_registry.parts").df()
        except duckdb.duckdb.BinderException:
            df_parts = duckdb.sql(f"SELECT * FROM igem_parts_registry.parts").df()
    return df_parts

def query_part(query):
    condition = df_parts["part_name"] == query
    return df_parts[condition]

def main(query_name, df_parts, G):

    result = query_part(query_name)
    assert len(result) == 1
    query_id = str(result['part_id'].values[0])
    # Get all first-degree connections (neighbors)
    first_degree_connections = list(G.neighbors(str(query_id)))
    subgraph = G.subgraph(first_degree_connections + [query_id])
    
    # Create a new Pyvis network
    net = Network(notebook=False)
    
    # Add the subgraph to the Pyvis network
    net.from_nx(subgraph)
    
    # Draw the graph to an HTML file
    return net

In [ ]:
graphml_path="../data/interim/parts_network.graphml"
df_parts = load_parts_table(sqlite_path="../data/raw/igem_parts_registry.sqlite")
G = nx.read_graphml(graphml_path)

In [ ]:
query_name = "BBa_K3085004"
net = main(query_name, df_parts, G)
net.show('subgraph.html', notebook=False)

In [ ]:
load_file(sqlite_path="../data/raw/igem_parts_registry.sqlite")

In [ ]:
%%bash
ls -lah duck.db